## ToDo
- Increase number of videos
- Protocol to run once/hour
- Only save new data (not duplicates)
- Look through the TikTok API documentation to see if can reach back further
    - Seems like no
- Increase video count

In [1]:
import pandas as pd
import pyktok as pyk
import random
import time

If pyktok does not operate as expected, you may find it helpful to run the 'specify_browser' function. 'specify_browser' takes as its sole argument a string representing a browser installed on your system, e.g. "chrome," "firefox," "edge," etc.


## Hourly pulls of user videos

#### Issues:

In [2]:
influencers = pd.read_csv('usernames.csv')
random.seed(11)
influencer_sample = random.sample(list(influencers.Username), 100)

In [3]:
for n in range(12):
    for influencer in influencer_sample:
        attempt = 0
        num_failed = 0
        while attempt < 5:
            try:
                pyk.save_tiktok_multi_page(influencer,ent_type='user',save_video=False,metadata_fn='9-9-influencer-metadata-hourly.csv')
                attempt = 0
                break
            except Exception as e:
                attempt += 1
                print(f"Attempt {attempt} failed: {e}")
                time.sleep(5)
                if attempt >= 5:
                    print("Max retries reached. Exiting.")
                    attempt = 0
                    num_failed += 1
                    break
    df = pd.read_csv('9-9-influencer-metadata-hourly.csv')
    df_cleaned = df.drop_duplicates(subset="video_id")
    df_cleaned.to_csv('9-9-influencer-metadata-hourly.csv')
    print('sleeping for 10 minutes ;)')
    print(f'number of failed videos: {num_failed}')
    time.sleep(600)

Resulting dataset: 9-9-influencer-metadata-hourly.csv

## Hourly pulls of related videos

#### Issues:
- Related videos only pulled from the same user when running ~hourly

In [6]:
influencer_videos = pd.read_csv('9-9-influencer-metadata-hourly.csv')

In [16]:
video_sample = random.sample(list(influencer_videos[['video_id', 'author_username']].itertuples()), 10)

In [ ]:
for n in range(6):
    for video in video_sample:
        attempt = 0
        num_failed = 0
        while attempt < 5:
            try:
                pyk.save_tiktok_multi_page(f'https://www.tiktok.com/@{video.author_username}/video/{video.video_id}',ent_type='video_related',save_video=False,metadata_fn=f'related-videos/9-13-rv-{video.video_id}.csv')
                attempt = 0
                break
            except Exception as e:
                attempt += 1
                print(f"Attempt {attempt} failed: {e}")
                time.sleep(5)
                if attempt >= 5:
                    print("Max retries reached. Exiting.")
                    attempt = 0
                    num_failed += 1
                    break
        df = pd.read_csv(f'related-videos/9-13-rv-{video.video_id}.csv')
        df_cleaned = df.drop_duplicates(subset="video_id")
        df_cleaned.to_csv('related-videos/9-13-rv-{video_id}.csv')
    print('sleeping for 10 minutes ;)')
    print(f'number of failed videos: {num_failed}')
    time.sleep(600)

Outputs: realted-videos/

In [ ]:
pyk.save_tiktok_multi_page(f'https://www.tiktok.com/@rkalimann/video/7374602041613864198',ent_type='video_related',save_video=False,metadata_fn=f'related-videos/9-13-rv-singletest.csv')

Output: related-videos/9-13-rv-singletest.csv

## Increase number of videos

In [ ]:
pyk.save_tiktok_multi_page(f'https://www.tiktok.com/@rkalimann/video/7374602041613864198', video_ct=50, ent_type='video_related',save_video=False,metadata_fn=f'related-videos/9-13-rv-singletest.csv')

Output: related-videos/9-13-rv-singletest.csv

Returned 60 videos

Suggestions:
- Potentially adding a prevent duplicates argument in pyktok? Can manually remove duplicates, but would be more efficient to prevent pyktok to skip videos it has already scraped